# Convert to geometry

In [ ]:
-- Add a new column for GEOGRAPHY type
ALTER TABLE `project_id.dataset_name.table_name` 
ADD COLUMN geo_gg GEOGRAPHY;

-- Populate the new column with geometry data
UPDATE `project_id.dataset_name.table_name` 
SET geo_gg = ST_GEOGFROMTEXT(geometry) 
WHERE geometry IS NOT NULL;

# Query

## n(HH) by Veh by MPO

In [ ]:
SELECT
  polygons_table.mpo AS mpo,
  points_table.vehicles AS vehicles,
  COUNT(DISTINCT household_id) AS num_households
FROM
  `project-using-replica.tdm_calib23.mpo_geo` AS polygons_table
JOIN
  `replica-customer.northeast.northeast_2023_Q4_population` AS points_table
ON
  ST_CONTAINS(polygons_table.geo_gg, ST_GEOGPOINT(points_table.lng, points_table.lat))
WHERE 
  points_table.lat IS NOT NULL
GROUP BY
  polygons_table.mpo, points_table.vehicles;

## n(HH) by VA by MPO

> cannot get VA?